# A Simple Decoder

Karime Maamari

---

This exercise is concerned with the "decompilation" of some set of the following input instructions:
1. **PUSH N:** Pushes the given integer, N, onto the stack
2. **ADD:** Pops the two top values from the stack, adds them, and pushes the result
3. **SUB:** Pops the two top values from the stack, *subtracts* them, and pushes the result
4. **MUL:** Pops the two top values from the stack, *multiplies* them, and pushes the result
5. **SWAP:** Swaps the top two numbers from the stack

As an example, consider the following set of commands:
1. PUSH 1
2. PUSH 2
3. PUSH 3
4. MUL
5. ADD

First, we push the values 1, 2, and 3 onto the stack

In [1]:
stack = ["3","2","1"]
for instruction in stack:
    print("----\n",instruction,"\n----\n")

----
 3 
----

----
 2 
----

----
 1 
----



Then MUL pops off the top two terms and pushes their product onto the stack

In [2]:
stack = ["6","1"]
for instruction in stack:
    print("----\n",instruction,"\n----\n")

----
 6 
----

----
 1 
----



And, finally, ADD pops off these two values and pushes their sum back onto the stack

In [3]:
print("----\n",7,"\n----\n")

----
 7 
----



Continue to the input handler (two cells below) to use the decompiler

---

### The Decompiler class

In [318]:
class Decompiler:
    
    def __init__(self):
        """
        Object initialization
        
        Parameters:
        -----------
        stack: arr
            The stack object to be manipulated
        count: int
            The variable identifier, instead of using different variable names, we use an identifier
            Ex: x0, x1, x2, etc.
        """
        
        self.stack = []
        self.count = 0
        self.alphabet = list("xyzabcdefghijklmnopqrstuvw")
        
    def push(self, element):        
        """
        Pushes element into stack
        
        Parameters:
        -----------
        element: int
            value to be placed onto the stack
        """
        
        self.stack.append(str(element))
        
    def pop(self):
        """
        Pops element from stack
        """
        
        if len(self.stack):
            return self.stack.pop()
        
    def operationHelper(self, operation):
        """
        Helper function for operations.
        All operations require the general logic:
            1. If two or more terms exist, operate between the terms
            2. If one term exists, operate between term and variable
            3. If no terms exist, operate between two variables
        So we can generalize, feeding in the operation type as input
        
        Parameters:
        -----------
        operation: string
            Type of operation ("+", "-", or "*")
        """
        
        stackLength = len(self.stack)
        
        # Normal operation between two terms
        if stackLength >= 2:
            firstTerm = self.pop()
            secondTerm = self.pop()
            self.push("("+firstTerm+"{}".format(operation)+secondTerm+")")
        
        # Only one term on stack
        elif stackLength == 1:
            firstTerm = self.pop()
            self.push("("+self.alphabet[self.count]+"{}".format(operation)+firstTerm+")")
            self.count= self.count + 1
            
        # No terms on stack
        else:
            self.push("("+self.alphabet[self.count]+"{}".format(operation)+self.alphabet[self.count+1]+")")
            self.count+=2
        
    def add(self):
        """
        Addition operation
        """
        
        self.operationHelper("+")
            
    def sub(self):
        """
        Subtraction operation
        """
        
        self.operationHelper("-")
            
    def mul(self):
        """
        Multiplication operation
        """

        self.operationHelper("*")
            
    def swap(self):
        """
        Swap the 2 elements in the stack
        """
        
        if len(self.stack) >= 2:
            firstTerm = self.pop()
            secondTerm = self.pop()
            self.push(firstTerm)
            self.push(secondTerm)
            
    def expression(self,output=False):
        """
        Returns:
        -------
        The expression based on the stack
        """
        if output: print(self.stack[0])
        return self.stack[0]

    def simplify(self):
        """
        Returns:
        -------
        
        """
        
        # Use sympy as sanity check
        from sympy import sympify as sym

        string = list(self.expression())        
        
        i = len(string)-1
        while i>=0:
            if string[i]=='(':
                # Subtraction
                if string[i-1]=='-':
                    k=i+2
                    while string[k]!=')':
                        if string[k]=='+':
                            string[k]='-'
                        elif string[k]=='-':
                            string[k]='+'  
                        k+=2
                
                # Multiplication
                if string[i-1]=='*':
                    k=i+2
                    while string[k]!=')':
                        if string[k]=='+' or string[k]=='-':
                            string[k+1]=string[i-2]+'*'+string[k+1]
                        k+=2
                
                # Delete parenthesis pairing
                del string[i]
                j=i
                while j<len(string):
                    if string[j]==')':
                        del string[j]
                        break
                    j+=1          
            i-=1
            
        # Int multiplication simplification
        intExists=0
        i=0
        while i<len(string)-1:
            if string[i].isdigit() and (string[i+1]=='*'):
                firstIntIndex=i
                intExists=1
                break 
            i+=1
        
        # Delete *1 cases
        j=0
        while j<len(string):
            deletionFlag=0
            if string[j].isdigit():
                if string[j-1]=='*':
                    if int(string[j])==1:
                        del string[j-1]
                        del string[j-1]
                        deletionFlag=1

            j+=1
            if deletionFlag: j-=2
                    
        if intExists:
            intSum=int(string[firstIntIndex])
            intIndexer=firstIntIndex+1
            
            while intIndexer<len(string):
                deletionFlag=0
                if string[intIndexer].isdigit():
                    if string[intIndexer-1]=='*':
                        if int(string[intIndexer])!=1:
                            intSum*=int(string[intIndexer])
                        del string[intIndexer-1]
                        del string[intIndexer-1]
                        deletionFlag=1

                string[firstIntIndex]=str(intSum)
                intIndexer+=1
                if deletionFlag: intIndexer-=2
        
        # Int addition/subtraciton simplification
        intExists=0
        i=0
        while i<len(string)-1:
            if string[i].isdigit() and (string[i+1]=='+' or string[i+1]=='-'):
                if i==0:
                    firstIntIndex=i
                    intExists=1
                    break 
                if i!=0 and (string[i-1]=='+' or string[i-1]=='-'):
                    firstIntIndex=i
                    intExists=1
                    break
            i+=1
        
        if intExists:
            intSum=int(string[firstIntIndex])
            intIndexer=firstIntIndex+1
            
            while intIndexer<len(string):
                deletionFlag=0
                if string[intIndexer].isdigit():
                    if string[intIndexer-1]=='+' and (intIndexer==len(string)-1 or string[intIndexer+1]!='*'):
                        intSum+=int(string[intIndexer])
                        del string[intIndexer-1]
                        del string[intIndexer-1]
                        deletionFlag=1
                    elif string[intIndexer-1]=='-' and (intIndexer==len(string)-1 or string[intIndexer+1]!='*'):
                        intSum-=int(string[intIndexer])
                        del string[intIndexer-1]
                        del string[intIndexer-1]
                        deletionFlag=1

                string[firstIntIndex]=str(intSum)
                intIndexer+=1
                if deletionFlag: intIndexer-=2
                
        # Delete +/-0 cases
        j=0
        while j<len(string):
            deletionFlag=0
            if string[j].isdigit():
                if int(string[j])==0:
                    if j==0 and (string[j+1]=='-' or string[j+1]=='+'):
                        del string[j]
                        del string[j]
                        deletionFlag=1
                    elif j==len(string)-1 and (string[j-1]=='-' or string[j-1]=='+'):
                        del string[j-1]
                        del string[j-1]
                        deletionFlag=1
                    elif (string[j-1]=='-' or string[j-1]=='+') and (string[j+1]=='-' or string[j+1]=='+'):
                        del string[j-1]
                        del string[j-1]
                        deletionFlag=1
            j+=1
            if deletionFlag: j-=2
        
        # Organize order
        indx = len(string)-2
        while indx>0:
            if string[indx]=='*' and string[indx+1].isdigit() and not string[indx-1].isdigit():
                string[indx-1], string[indx+1] = string[indx+1], string[indx-1] 
        
        return (sym(self.expression()),''.join(string))

### The input handler

In [ ]:
# Test 9: Order of multiplication terms
s = Decompiler()
s.push(2)
s.mul()
s.mul()
s.expression(output=True)
s.simplify()

(y*(x*2))


In [319]:
s = Decompiler()
print("Enter operation name:\n")
print("  PUSH <N>: Push an integer, N, to the stack")
print("  ADD: Add values from the stack")
print("  SUB: Subtract values from the stack")
print("  MUL: Multiply values from the stack")
print("  SWAP: Swap values from the stack")
print("  END: Enter END when finished with commands\n")

while True:
    operation = input().split()
    try:
        if len(operation)>1 and operation[0].upper()!="PUSH":
            print("Error, please limit entries to one operation per line")
            continue
        if operation[0].upper() == "PUSH":
            valueToPush = operation[1]
            s.push(int(valueToPush))
        elif operation[0].upper() == "ADD":
            s.add()
        elif operation[0].upper() == "SUB":
            s.sub()
        elif operation[0].upper() == "MUL":
            s.mul()
        elif operation[0].upper() == "SWAP":
            s.swap()
        elif operation[0].upper() == "END":
            break      
        else:
            print("Error, please enter a valid operation:\n  PUSH <n>, ADD, SUB, MUL, or SWAP")
    except:
        print("Error, please enter a single keyword unless pushing, in which case enter the keyword \"PUSH\" followed by an integer")

if len(s.stack):
    print(s.simplify())

Enter operation name:

  PUSH <N>: Push an integer, N, to the stack
  ADD: Add values from the stack
  SUB: Subtract values from the stack
  MUL: Multiply values from the stack
  SWAP: Swap values from the stack
  END: Enter END when finished with commands

end


---

### Test cases

In [308]:
# Test 1: 1+2=3
s = Decompiler()
s.push(1)
s.push(2)
s.add()
s.expression(output=True)
s.simplify()

(2+1)


(3, '3')

In [309]:
# Test 2: 1+2*3=7
s = Decompiler()
s.push(1)
s.push(2)
s.push(3)
s.mul()
s.add()
s.expression(output=True)
s.simplify()

((3*2)+1)


(7, '7')

In [310]:
# Test 3: 1+2*3=7
s = Decompiler()
s.push(2)
s.push(3)
s.mul()
s.push(1)
s.add()
s.expression(output=True)
s.simplify()

(1+(3*2))


(7, '7')

In [311]:
# Test 4: 1*2*x
s = Decompiler()
s.push(2)
s.mul()
s.push(1)
s.add()
s.expression(output=True)
s.simplify()

(1+(x*2))


(2*x + 1, '1+2*x')

In [312]:
# Test 5: x-1
s = Decompiler()
s.push(1)
s.sub()
s.expression(output=True)
s.simplify()

(x-1)


(x - 1, 'x-1')

In [313]:
# Test 6: 2+1*3=5
s = Decompiler()
s.push(1)
s.push(2)
s.swap()
s.push(3)
s.mul()
s.add()
s.expression(output=True)
s.simplify()

((3*1)+2)


(5, '5')

In [314]:
# Test 7: z+(x*y)
s = Decompiler()
s.mul()
s.add()
s.expression(output=True)
s.simplify()

(z+(x*y))


(x*y + z, 'z+x*y')

In [315]:
# Test 8: 1
s = Decompiler()
s.push(1)
s.push(2)
s.push(3)
s.expression(output=True)
s.simplify()

1


(1, '1')

In [317]:
# Test 10: Empty simplify: Returns error, but in practice, the input handler would catch this
s = Decompiler()
# s.expression(output=True)
# s.simplify()

In [261]:
# Test 11: Empty swap: Returns error, but in practice, input handler would catch this
s = Decompiler()
# s.swap()
# s.expression(output=True)
# s.simplify()